# Apartat A

In [1]:
import ast

import pandas as pd
import numpy as np

from sklearn import linear_model, svm, datasets
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn import neighbors, tree, naive_bayes
from sklearn.metrics import f1_score
from sklearn.preprocessing import MinMaxScaler


import scipy

import matplotlib.pyplot as plt

import seaborn as sns

In [2]:
# EDA
#41 Atributs
#tipus definits a baix
#El target és un tipus int (0,1) depenen de si és llegendari o no (dos classes)
#taula de correlacions amb els valors de baix
#histogrames a baix, majoria balancejades pero l'atribut objectiu hi ha pocs de classe 1 (llegendaris)

#muy pocos legendarios, estudiamos datasets con igual numero de pokemons legendarios y no legendarios o todos juntos? pq tambien es importante que existen pocos legendarios


**name**: The English name of the Pokemon\
**japanese_name**: The Original Japanese name of the Pokemon\
**pokedex_number**: The entry number of the Pokemon in the National Pokedex\
**percentage_male**: The percentage of the species that are male. Blank if the Pokemon is genderless.\
**type1**: The Primary Type of the Pokemon\
**type2**: The Secondary Type of the Pokemon\
**classification**: The Classification of the Pokemon as described by the Sun and Moon Pokedex\
**height_m**: Height of the Pokemon in metres\
**weight_kg**: The Weight of the Pokemon in kilograms\
**capture_rate**: Capture Rate of the Pokemon\
**baseeggsteps**: The number of steps required to hatch an egg of the Pokemon\
**abilities**: A stringified list of abilities that the Pokemon is capable of having\
**experience_growth**: The Experience Growth of the Pokemon\
**base_happiness**: Base Happiness of the Pokemon\
**against_?**: Eighteen features that denote the amount of damage taken against an attack of a particular type\
**hp**: The Base HP of the Pokemon\
**attack**: The Base Attack of the Pokemon\
**defense**: The Base Defense of the Pokemon\
**sp_attack**: The Base Special Attack of the Pokemon\
**sp_defense**: The Base Special Defense of the Pokemon\
**speed**: The Base Speed of the Pokemon\
**generation**: The numbered generation which the Pokemon was first introduced\
**is_legendary**: Denotes if the Pokemon is legendary.

In [3]:
df = pd.read_csv('pokemon.csv')

In [4]:
TARGET = 'is_legendary'
FILTER_RATE = 0.1
NAMES = df['name']

### APARTAT A

**EXPLORATORY DATA ANALYSIS**

**Quants atributs té la vostra base de dades?**

41 atributs

In [5]:
len(df.columns)

41

**Quins tipus d'atributs tens?**

In [6]:
df.dtypes.unique()

array([dtype('O'), dtype('float64'), dtype('int64')], dtype=object)

**Com és el *target*, quantes categories diferents existeixen?**

In [7]:
df[TARGET].unique()  # Atribut binari

array([0, 1])

**Podeu veure alguna correlació entre X i y?**

In [8]:
df.corr()[TARGET].abs().sort_values(ascending=False)

is_legendary         1.000000
base_egg_steps       0.873488
base_total           0.485440
base_happiness       0.413108
sp_attack            0.406281
weight_kg            0.393023
experience_growth    0.361038
sp_defense           0.343241
height_m             0.322155
speed                0.311639
hp                   0.308405
attack               0.303295
defense              0.265587
pokedex_number       0.196785
against_ghost        0.170746
generation           0.139029
against_dark         0.136315
against_psychic      0.106047
against_grass        0.070826
against_flying       0.062214
against_fight        0.059132
against_fairy        0.050165
percentage_male      0.045222
against_normal       0.034761
against_bug          0.027864
against_poison       0.024349
against_electric     0.023151
against_water        0.020679
against_rock         0.017588
against_dragon       0.014844
against_ground       0.012812
against_fire         0.011073
against_ice          0.005580
against_st

**Estan balancejades les etiquetes?**

In [9]:
# hist = df.hist(figsize=(16, 16))

In [10]:
df[TARGET].value_counts()

0    731
1     70
Name: is_legendary, dtype: int64

**PREPROCESSING**

**Com controleu les dades sense informació (NaNs)?**

In [11]:
df.isnull().sum()
#df.shape

abilities              0
against_bug            0
against_dark           0
against_dragon         0
against_electric       0
against_fairy          0
against_fight          0
against_fire           0
against_flying         0
against_ghost          0
against_grass          0
against_ground         0
against_ice            0
against_normal         0
against_poison         0
against_psychic        0
against_rock           0
against_steel          0
against_water          0
attack                 0
base_egg_steps         0
base_happiness         0
base_total             0
capture_rate           0
classfication          0
defense                0
experience_growth      0
height_m              20
hp                     0
japanese_name          0
name                   0
percentage_male       98
pokedex_number         0
sp_attack              0
sp_defense             0
speed                  0
type1                  0
type2                384
weight_kg             20
generation             0


**Estan les dades normalitzades? Caldria fer-ho?**

In [12]:
for col in df.select_dtypes(exclude='object'):
    stat, pvalue = scipy.stats.normaltest(df[col])
    if pvalue > 1e-3:
        print(col)

In [13]:
# 2. Preprocessing
#No estàn normalitzades, 
#No té sentit intentar normalitzar ja que la majoria de dades que tenim són binàries
#No tenim gaires nan's excepte en el type2, no podem reomplir-les degut a que són strings
# (faltar contestar mes coses aqui...) Canvis dataset: En la columna capture_rate hi ha un valor que és de tipus object que no té sentit, hem ficat el valor de 70 al buscar el capture rate del pokemon a internet
# Tenim algunes variables categòriques com: classification, type1, type2, name i japanese_name, abilities (lista)
#

In [14]:
df.at[df['name'] == 'Minior', 'capture_rate'] = 70  # Because '30 (Meteorite)255 (Core)'
df['capture_rate'] = df['capture_rate'].astype('int64')

In [15]:
df.describe().loc[['mean', '50%', 'std']]

,against_bug,against_dark,against_dragon,against_electric,against_fairy,against_fight,against_fire,against_flying,against_ghost,against_grass,...,height_m,hp,percentage_male,pokedex_number,sp_attack,sp_defense,speed,weight_kg,generation,is_legendary
mean,0.996255,1.057116,0.968789,1.073970,1.068976,1.065543,1.135456,1.192884,0.985019,1.034020,...,1.163892,68.958801,55.155761,401.000000,71.305868,70.911361,66.334582,61.378105,3.690387,0.087391
50%,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,65.000000,50.000000,401.000000,65.000000,66.000000,65.000000,27.300000,4.000000,0.000000
std,0.597248,0.438142,0.353058,0.654962,0.522167,0.717251,0.691853,0.604488,0.558256,0.788896,...,1.080326,26.576015,20.261623,231.373075,32.353826,27.942501,28.907662,109.354766,1.930420,0.282583


In [16]:
# Gaussian if (mediana y media similar y desv.std entre el 20 y 30% de la media)
desc = df.describe().loc[['mean', '50%', 'std']]

In [17]:
#atributs que no es poden normalitzar: generation, is_legendary, els que són objectes

**Com codifiqueu les dades categòriques? (OrdinalEncoder, OneHotEncoder)**

In [18]:
df_2 = df.copy(deep=True)
df_2['abilities'] = df['abilities'].apply(ast.literal_eval)
df_2[df_2['abilities'].explode().unique()] = 0

for idx, row in df_2.iterrows():
    for cat in row['abilities']:
        df_2.at[idx, cat] = 1

df_2.drop(columns='abilities', inplace=True)
df_2 = pd.get_dummies(df_2, columns=['classfication', 'type1', 'type2'], prefix=['', '1', '2'], prefix_sep='')
df_2.drop(columns=['name', 'japanese_name'], inplace=True)

**Quin tipus de normalització serà més adient per les vostres dades?**

In [19]:
cols = df_2.columns
scaler = MinMaxScaler()
df_3=scaler.fit_transform(df_2) #dataset escalat
df_3 = pd.DataFrame(df_3,
columns=cols)

In [40]:
corr = df_3.corr().abs()[TARGET].sort_values(ascending=False)
corr = corr[corr > FILTER_RATE]
#corr
df_4=df_3[corr.index]
df_4 #dataset amb atributs filtrats per correlacio
# corr.index
# df_4.columns[df_4.columns.duplicated()]
# corr.index.unique()


,is_legendary,base_egg_steps,base_total,base_happiness,sp_attack,weight_kg,experience_growth,sp_defense,capture_rate,height_m,...,Paleozoic Pokémon,Iron Will Pokémon,Deep Black Pokémon,Cyclone Pokémon,Grassland Pokémon,Cavern Pokémon,1dragon,against_psychic,1steel,Clear Body
0,0.0,0.130435,0.230000,0.5,0.298913,0.006801,0.442173,0.214286,0.166667,0.041667,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.500,0.0,0.0
1,0.0,0.130435,0.375000,0.5,0.380435,0.012903,0.442173,0.285714,0.166667,0.062500,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.500,0.0,0.0
2,0.0,0.130435,0.741667,0.5,0.608696,0.099920,0.442173,0.476190,0.166667,0.131944,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.500,0.0,0.0
3,0.0,0.130435,0.215000,0.5,0.271739,0.008402,0.442173,0.142857,0.166667,0.034722,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.250,0.0,0.0
4,0.0,0.130435,0.375000,0.5,0.380435,0.018904,0.442173,0.214286,0.166667,0.069444,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.250,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
796,1.0,1.000000,0.650000,0.0,0.527174,1.000000,0.625000,0.385714,0.087302,0.631944,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.125,1.0,0.0
797,1.0,1.000000,0.650000,0.0,0.266304,0.000000,0.625000,0.052381,1.000000,0.013889,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.125,0.0,0.0
798,1.0,1.000000,0.650000,0.0,0.472826,0.888078,0.625000,0.157143,0.047619,0.375000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0
799,1.0,1.000000,0.700000,0.0,0.635870,0.229946,0.625000,0.328571,0.000000,0.159722,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.125,0.0,0.0


**Caldra aplicar un PCA?**

**Es poden aplicar `PolynomialFeatures` per millorar la classificació?**

No perquè hi ha massa atributs. Els `PolynomialFeatures` aumenten exponencial.

**MODEL SELECTION**

**Quins models heu considerat?**

In [46]:
# df_4.isnull().sum()[df_4.isnull().sum() > 0]
#valor=mean(df_4['weight_kg'])
# df_4[np.isnull(df_4)] = df_4.mean()
#df_4.describe().loc[['mean', '50%', 'std']]
df_4 = df_4.fillna(df_4.mean())

In [47]:
df_4.isnull().sum()[df_4.isnull().sum() > 0]

Series([], dtype: int64)

In [135]:
X=df_4.copy(deep=True)
X.drop(columns=TARGET, inplace=True)
y=df_4[TARGET]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

classifiers = [
    LogisticRegression(max_iter=10000),
    svm.SVC(kernel='linear', C=1.0),
    svm.LinearSVC(C=1.0, max_iter=1000000),
    svm.SVC(kernel='rbf', gamma=0.7, C=1.0),
    svm.SVC(kernel='poly', degree=3, gamma='auto', C=1.0),
    linear_model.SGDClassifier(loss="modified_huber", alpha=0.1),
    neighbors.KNeighborsClassifier(n_neighbors=15),
    tree.DecisionTreeClassifier(),
    naive_bayes.GaussianNB()
]

scores = {}

for classifier in classifiers:
    classifier.fit(X_train, y_train)
    y_pred = classifier.predict(X_test)
    score = classifier.score(X_test, y_test)
    print(f'{classifier.__class__.__name__}: {score}')
    scores[classifier.__class__.__name__] = score

# scores

LogisticRegression: 0.9751552795031055
SVC: 0.9875776397515528
LinearSVC: 0.9875776397515528
SVC: 0.9875776397515528
SVC: 0.906832298136646
SGDClassifier: 0.9440993788819876
KNeighborsClassifier: 0.906832298136646
DecisionTreeClassifier: 1.0
GaussianNB: 0.9254658385093167


In [64]:
len(classifiers[0].coef_[0])

104

In [126]:

results=[]
for classifier in classifiers:
    # print(classifier.__class__.__name__)
    try:
        results.append(dict(zip(X.columns, classifier.coef_[0])))
    except:
        pass
len(results)

4

In [134]:
# coeficients = dict.fromkeys(X.columns, [])

coeficients = {col: [] for col in X.columns}

for classifier in classifiers:
    try:
        for col, coef in zip(coeficients, classifier.coef_[0]):
            coeficients[col].append(coef)
    except:
        pass

coeficients

# len(classifiers)

{'base_egg_steps': [2.343095622221887,
  1.6995309569257309,
  1.2332753709251518,
  0.7335458618878962],
 'base_total': [0.7843445762563632,
  0.24454695845394103,
  0.07657111535222398,
  0.2976867251606458],
 'base_happiness': [-0.2488785894958053,
  0.3124622150763978,
  0.11459199026172012,
  0.034580574909545386],
 'sp_attack': [0.41865923039071284,
  0.2666218147193178,
  0.05289569905355108,
  0.1848701610452881],
 'weight_kg': [0.5646203736563619,
  0.252022797785952,
  0.35808009516907935,
  0.17612004963894756],
 'experience_growth': [0.7991315870447183,
  0.37425559025794386,
  -0.018305397396404185,
  0.3117146063809838],
 'sp_defense': [0.37487461162292507,
  0.17169343184636032,
  -0.008100851161285741,
  0.16685727520783203],
 'capture_rate': [-1.084219554233543,
  -0.6245006900103648,
  -0.6742374027868133,
  -0.18172015976442668],
 'height_m': [0.12845814077610523,
  -0.04394515380809216,
  -0.04770372312426204,
  0.05750045072776399],
 'speed': [0.5951997426276332,
 

**Considereu les SVM amb els diferents *kernels* implementats.**

**Quin creieu que serà el més precís?**

**Quin serà el més ràpid?**

**Seria una bona idea fer un `ensemble`? Quins inconvenients creieu que pot haver-hi?**

La tasca d'aquesta pràctica s'enmarca dins l'aprenentatge computacional supervisat. A sklearn, disposem de varies tècniques (veure documentació). A les classes de teoria, hem vist varies tècniques, com ara logistic regression, SVM amb diferents kernels, Nearest Neighbour, i el perceptró... En aquesta secció heu de valorar quina o quines tècniques voleu fer servir, aixi com també explicar el per què les heu seleccionat. Recomanem, que per entendre millor la teoria, s'ha de provar com a mínim un model de SVM.

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=c3c3f0e5-53b5-41aa-b55f-8ff8474beec2' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>